# PCA and KernelPCA
Here is the implementation of PCA and KPCA using PyOD and Scikit-learn for Anomaly Detection


```
References PCA,KPCA: 
1. Hoffman, Kernel PCA as Novelty Detecion: https://www.sciencedirect.com/science/article/abs/pii/S0031320306003414
2. http://videolectures.net/ecmlpkdd08_lazarevic_dmfa/
3. http://videolectures.net/kdd2010_krogel_odt/
4. https://pyod.readthedocs.io/en/latest/
5. https://scikit-learn.org/stable/
6. https://github.com/Nmerrillvt/kPCA
7. https://github.com/Quantumgunhee/VQOCC
8. https://github.com/GuansongPang/ADRepository-Anomaly-detection-datasets/tree/main
9. https://www.heikohoffmann.de/kpca.html

```

<font color='white'> Define **functions** </font>

In [1]:
# Import the necessary libraries.
import pandas as pd
import numpy as np
from numpy import reshape
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import seaborn as sns

!pip install -q pyod            
# Install PyOD. The PyOD library provides several outlier detection methods that can be used
# !pip install --upgrade pyod  
# or update if needed

from pyod.models.pca import PCA as PCA_PYOD
from pyod.models.kpca import KPCA as KPCA_PYOD

from sklearn.decomposition import PCA, KernelPCA
from sklearn.linear_model import LogisticRegression

from pyod.utils.data import evaluate_print
from pyod.utils.example import visualize

from sklearn.model_selection import train_test_split
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix, f1_score, accuracy_score, precision_score, recall_score, precision_recall_curve, classification_report, roc_curve, auc, roc_auc_score

from sklearn.datasets import load_iris, load_breast_cancer

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

from collections import Counter

**Outlier Detection** based on PyOD library and Hoffman's Novelty Detection

FUNCTIONS PyOD:  PCA & Kernel-PCA Outlier Detection:
Train the model with normal and abnormal data. Then trying a test set and  making decision if it is outlier or not based on Hoffmans Paper for Novelty Detection and PyOD documentation.

In [2]:
def pca_kpca_with_metrics(model, X_train, y_train, X_test, y_test, contamination=0.1,
                            kernel='rbf', gamma=0.1, degree=3, coef0=1):

  if model=='pca':
    # Fit the model
    model_ad = PCA_PYOD(n_components=None, contamination=contamination)
    model_ad.fit(X_train,y=None)

    n_components = model_ad.n_components_
    components = model_ad.components_

    print("The non-0s PCA components are: ",n_components)
    # print("The components: \n", components)

    # exp_var = model_ad.explained_variance_
    exp_var_rat = model_ad.explained_variance_ratio_

    plot_explained_variance(exp_var_rat)
    n_components = explained_variance_threshold(exp_var_rat, n_components)

    model_ad = PCA_PYOD(n_components=n_components, contamination=contamination)
  else: 
    if model=='kpca':
      n_components=2
      model_ad = KPCA_PYOD(n_components=n_components, kernel=kernel, gamma=gamma, degree=degree, coef0=coef0, contamination=contamination)   
    else:
      print("wrong input model")
      return
  
  model_ad.fit(X_train,y=None)

  threshold_model_ad = model_ad.threshold_
  print("The threshold of the PCA method for the given contamination rate:" , threshold_model_ad)

  y_train_pred = model_ad.predict(X_train)
  y_train_scores= model_ad.decision_function(X_train)

  y_test_pred = model_ad.predict(X_test) # outlier labels (0 or 1)
  y_test_scores = model_ad.decision_function(X_test) #outlier scores

  print("\nTrain-set with the threshold:\n")
  plot_anomalies_with_threshold(y_train_scores, threshold_model_ad)
  print("Another option to choose better threshold from the above plot.\nThen run again for other contamination for better results.\n")
  print("\nTest-set with the threshold:\n")
  plot_anomalies_with_threshold(y_test_scores, threshold_model_ad)

  print(descriptive_stat_threshold(X_train, y_train_scores, threshold_model_ad))
  print("\n")
  print(descriptive_stat_threshold(X_test, y_test_scores, threshold_model_ad))
  print("\n")

  y_test_prob = model_ad.predict_proba(X_test, method='linear', return_confidence=False)[:, 1]

  calculate_metrics(y_test, y_test_scores, y_test_pred)
  calculate_metrics(y_train, y_train_scores, y_train_pred)

  print(evaluate_print('\nFor test data:', y_test, y_test_scores))

# Function to plot the explained variance ratio
def plot_explained_variance(exp_var_ratio):

  end = len(exp_var_ratio)+1
  plt.figure(figsize=(9, 6))
  plt.plot(range(1, end), exp_var_ratio*100)
  plt.xlabel('Principal Component ')
  plt.ylabel('Explained Variance percentage (%)')
  plt.xticks(range(1, end))
  plt.show()

# Function to plot the explained variance ratio with the selected cumulative %
def explained_variance_threshold(exp_var_ratio, number_components):

  cum_exp_var = np.cumsum(exp_var_ratio) * 100
  end = number_components+1
  plt.bar(range(1, end), exp_var_ratio * 100, align='center', label='Individual explained variance')
  plt.step(range(1, end), cum_exp_var, where='mid', label='Cumulative explained variance', color='red')
  plt.axhline(y=90, color='gray', linestyle='--', linewidth=1)
  plt.text(end, 90, '90%', color='gray', fontsize=10, va='center')
  plt.ylabel('Explained variance percentage (%)')
  plt.xlabel('Principal component')
  plt.legend(loc='best')
  plt.xticks(ticks=range(1, end))
  plt.tight_layout()
  plt.show()
  # The number of components with the pre-selected % of cumulative %
  # Maybe it is better to increase it to 95% instead of 90%
  number_components = np.where(cum_exp_var > 90)[0][0] + 1
  print("Number of principal components needed to explain 90% of variance:", number_components)

  # Returns the number of components
  return number_components

# Function that plots the histogram of anomaly scores of the test-set with the threshold
def plot_anomalies_with_threshold(y_scores, threshold):

  plt.hist(y_scores, bins=50) 
  plt.axvline(x=threshold, color='red', linestyle='--', linewidth=2)
  plt.text(threshold, max([list(y_scores).count(x) for x in set(y_scores)]),'Threshold', color='red', fontsize=10, va='center')
  plt.title("Histogram with the threshold")
  plt.xlabel('Anomaly score')
  plt.show()
  print("\n")

# Function that gives a table with statistics about the given test-set (Anomaly scores, Avg, count, count%)
def descriptive_stat_threshold(df, pred_score, threshold):

  df = pd.DataFrame(df)
  df['Anomaly_Score'] = pred_score
  df['Group'] = np.where(df['Anomaly_Score']<= threshold, 'Normal', 'Outlier')

  cnt = df.groupby('Group')['Anomaly_Score'].count().reset_index().rename(columns={'Anomaly_Score':'Count'})
  cnt['Count %'] = (cnt['Count'] / cnt['Count'].sum()) * 100 # The count and count %
  stat = df.groupby('Group').mean().round(2).reset_index() # The avg.
  stat = cnt.merge(stat, left_on='Group',right_on='Group') # Put the count and the avg. together
  return (stat)

# Function to plot the Confussion matrix for the given test-set
def conf_matrix(y_true, y_pred, labels=None, title='Confusion Matrix'):

  cm = confusion_matrix(y_true, y_pred, labels=labels)
  cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
  fig, ax = plt.subplots(figsize=(8, 6))
  disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
  disp.plot(include_values=True, cmap='Blues', ax=ax, xticks_rotation='horizontal')
  ax.set_title(title)
  ax.set_xlabel('Predicted Label')
  ax.set_ylabel('True Label')
  plt.show()
  return cm

# Function that plots the RocCurve for the given test-set
def roccurve(y_true, y_scores):

  fpr, tpr, thresholds = roc_curve(y_true, y_scores)
  auc_score = auc(fpr, tpr)
  plt.figure(figsize=(9, 6))
  plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {auc_score:.2f})')
  plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
  plt.xlabel('False Positive Rate')
  plt.ylabel('True Positive Rate')
  plt.title('Receiver Operating Characteristic')
  plt.legend(loc="lower right")
  plt.show()
  print("\n")

# Function that Prints and plots a lot of Metrics
def calculate_metrics(y_true, pred_scores, y_pred):

  # Confusion matrix
  print('Confusion Matrix:\n')
  cm = conf_matrix(y_true, y_pred)
  tn, fp, fn, tp = cm.ravel()
      
  # Calculate the AUC-ROC
  auc_roc = roc_auc_score(y_true, pred_scores)
  
  # ROC_Curve
  roccurve(y_true, pred_scores)

  accuracy = (tp+tn)/(tp+fp+tn+fn)
  print("Accuracy: ", accuracy)

  precision = (tp)/(tp+fp)
  print("Precision: ", precision)

  recall = (tp)/(tp+fn)
  print("Recall: ", recall)

  f1score = 2*(precision*recall)/(precision+recall)
  print("F1 Score: ", f1score)

  print(f'AUC-ROC: {auc_roc:.4f}')


**Novelty Detection** Hoffman's Novelty Detection

In [4]:
# TODO: A better decision for PCA method, the KernelPCA method is base on Hoffmans Novelty detection

def function_AD(model, 
                X_train, y_train, X_test, y_test, 
                n_components=None, 
                kernel='rbf', 
                gamma=None, 
                degree=3, 
                coef0=1, 
                contamination=0, 
                outlier=0,
                analysis=False):
  
  # Selection between PCA and KernelPCA
  if model=='pca':

    model_ad = PCA(n_components=None)                                         # Create the model for PCA
    model_ad.fit(X_train,y=None)                                              # Fit the model with the given train-set (the model should contain only the normal class)
    n_components = model_ad.n_components_                                     # The total number of components (For PCA are the number of the features)
    print("The non-0s PCA components are: \n",n_components)
    components = model_ad.components_                                         # The components
    exp_var = model_ad.explained_variance_                                    # Here is the explained variance for each componant
    exp_var_rat = model_ad.explained_variance_ratio_                          # Here is the explained variance ratio for each componant


    plot_explained_variance_AD(exp_var_rat)                                      # Here is the explained variance for each component
    n_components = explained_variance_threshold_AD(exp_var_rat, n_components)    # Select the components tha discribe the 90% of cumulative explained_variance

    # Re-fit the model with the selected components
    if n_components == 1: n_components = 2
    model_ad = PCA(n_components=n_components)

  else: 
    if model=='kpca':
      # TODO: The selected components for kpca is 2, but we can try a gridsearch in order to distinguish which number of components is better
      n_components=2

      
      model_ad = KernelPCA(n_components=n_components, kernel=kernel,          # Create the model
                           gamma=gamma, degree=degree, coef0=coef0, fit_inverse_transform=True)
    else:
      print("The model you can use are PCA and KernelPCA as: pca or kpca respectively")
      return

  # Fit the model with the X_train and transforming the X_train into the selected number of components and then reconstructed.
  X_train_reconstructed = model_ad.inverse_transform(model_ad.fit_transform(X_train))
  
  # Reduce and reconstruct the test-data back to its original 
  X_test_reduced = model_ad.transform(X_test)
  X_test_reconstructed = model_ad.inverse_transform(X_test_reduced)

  # Get the reconstruction Errors
  rec_errors_test = reconstruction_errors_AD(X_test, X_test_reconstructed)
  rec_errors_train = reconstruction_errors_AD(X_train, X_train_reconstructed)

  # Calculate the Novelty Scores 
  anom_scores = anomaly_scores_AD(rec_errors_train, rec_errors_test)

  # Threshold
  threshold = threshold_ratio_AD(X_train, y_test, rec_errors_train, outlier=outlier, contamination=contamination, anom_scores = anom_scores)

  # Classify the test data as normal or anomalous based on the reconstruction error and the selected threshold
  y_pred = [1 if e > threshold else 0 for e in rec_errors_test]

  if model=='pca' or analysis==False:

    # Plot the score of anomalies and the threshold
    plot_anomalies_with_threshold_AD(anom_scores, threshold)

    # Plot the true normal classes of the selected X_train into two Principal Components of the transformed Data
    plt.figure(figsize=(7, 6))
    plt.scatter(X_test_reduced[:, 0], X_test_reduced[:, 1], c=y_test, cmap='viridis')
    plt.xlabel('PC1')
    plt.ylabel('PC2')
    plt.title('The first 2 PC of the true test-set')
    plt.show()

    # Plot the predicted normals and the predicted anomalies of the selected X_test into two Principal Components of the transformed Data
    plt.figure(figsize=(7, 6))
    plt.scatter(X_test_reduced[:, 0], X_test_reduced[:, 1], c=y_pred, cmap='viridis')
    plt.xlabel('PC1')
    plt.ylabel('PC2')
    plt.title('The first 2 PC of the predicted test-set')
    plt.show()

    # Print informations about the given test-set
    descriptive_stat_threshold_AD(X_test, anom_scores, threshold)
    
    # Print and Plot all the Metrics
    calculate_metrics_AD(X_test, y_test, anom_scores, y_pred) 
    # print(f1_score(y_test, y_pred))
  else:
    return roc_auc_score(y_test, anom_scores), f1_score(y_test, y_pred)


    
# Function that calculates the reconstruction errors
def reconstruction_errors_AD(X, X_reconstructed):
  # Calculate the reconstruction error for each data point
  reconstruction_errors = np.mean((X - X_reconstructed) ** 2, axis=1)
  return reconstruction_errors

# Functions that returns the anomaly scores
def anomaly_scores_AD(reconstruction_errors_train, reconstruction_errors_test):
  # Calculate the Novelty Scores for each data point of the test-set
  mean_error = np.mean(reconstruction_errors_train)
  std_error = np.std(reconstruction_errors_train)
  normalized_error_test = (reconstruction_errors_test - mean_error) / std_error
  return np.abs(normalized_error_test)

# Function that returns the threshold
# Here I am not sure if is the correct way but we can find the best threshold for having the best F-1 score if the contamination=0. 
# Otherwise we can set it based a given threshold or as a contamination (% of outliers or anomalies in the test set)
def threshold_ratio_AD(X_train, y_true, reconstruction_errors, outlier=0, contamination=0, anom_scores = 0):
  if contamination==-1:

    # Calculate precision and recall for different thresholds
    precision, recall, thresholds = precision_recall_curve(y_true, anom_scores)

    # Calculate the F1 score for each threshold
    f1_scores = 2 * (precision * recall) / (precision + recall)
       
    # Find the index of the threshold that maximizes the F1 score
    optimal_idx = np.argmax(f1_scores)
       
    # Get the optimal threshold
    optimal_threshold = thresholds[optimal_idx]
    print("The optimum threshold of the PCA method:", optimal_threshold)
    threshold = optimal_threshold

  elif contamination==0:
    mean_error = np.mean(reconstruction_errors)
    std_error = np.std(reconstruction_errors)
    normalized_error = (reconstruction_errors - mean_error) / std_error
    threshold = mean_error + 2.5 * std_error

  elif contamination>0 and contamination<0.5:
    serr = np.sort(reconstruction_errors)
    threshold = serr[X_train.shape[0]-outlier-1]

  else:
    print("Not correct contamination ratio")

  return threshold

# Function to plot the explained variance ratio
def plot_explained_variance_AD(exp_var_ratio):

  end = len(exp_var_ratio)+1
  plt.figure(figsize=(9, 6))
  plt.plot(range(1, end), exp_var_ratio*100)
  plt.xlabel('Principal Component ')
  plt.ylabel('Explained Variance percentage (%)')
  plt.xticks(range(1, end))
  plt.show()

# Function to plot the explained variance ratio with the selected cumulative %
def explained_variance_threshold_AD(exp_var_ratio, number_components):

  cum_exp_var = np.cumsum(exp_var_ratio) * 100
  end = number_components+1
  plt.bar(range(1, end), exp_var_ratio * 100, align='center', label='Individual explained variance')
  plt.step(range(1, end), cum_exp_var, where='mid', label='Cumulative explained variance', color='red')
  plt.axhline(y=90, color='gray', linestyle='--', linewidth=1)
  plt.text(end, 90, '90%', color='gray', fontsize=10, va='center')
  plt.ylabel('Explained variance percentage (%)')
  plt.xlabel('Principal component')
  plt.legend(loc='best')
  plt.xticks(ticks=range(1, end))
  plt.tight_layout()
  plt.show()
  # The number of components with the pre-selected % of cumulative %
  # Maybe it is better to increase it to 95% instead of 90%
  number_components = np.where(cum_exp_var > 90)[0][0] + 1
  print("Number of principal components needed to explain 90% of variance:", number_components)

  # Returns the number of components
  return number_components

# Function that plots the histogram of anomaly scores of the test-set with the threshold
def plot_anomalies_with_threshold_AD(y_scores, threshold):

  plt.hist(y_scores, bins=50) 
  plt.axvline(x=threshold, color='red', linestyle='--', linewidth=2)
  plt.text(threshold, max([list(y_scores).count(x) for x in set(y_scores)]),'Threshold', color='red', fontsize=10, va='center')
  plt.title("Histogram with the threshold")
  plt.xlabel('Anomaly score')
  plt.show()
  print("\n")

# Function that gives a table with statistics about the given test-set (Anomaly scores, Avg, count, count%)
def descriptive_stat_threshold_AD(df, pred_score, threshold):

  df = pd.DataFrame(df)
  df['Anomaly_Score'] = pred_score
  df['Group'] = np.where(df['Anomaly_Score']<= threshold, 'Normal', 'Novelty')

  cnt = df.groupby('Group')['Anomaly_Score'].count().reset_index().rename(columns={'Anomaly_Score':'Count'})
  cnt['Count %'] = (cnt['Count'] / cnt['Count'].sum()) * 100 # The count and count %
  stat = df.groupby('Group').mean().round(2).reset_index() # The avg.
  stat = cnt.merge(stat, left_on='Group',right_on='Group') # Put the count and the avg. together
  return (stat)

# Function to plot the Confussion matrix for the given test-set
def conf_matrix_AD(y_true, y_pred, labels=None, title='Confusion Matrix'):

  cm = confusion_matrix(y_true, y_pred, labels=labels)
  cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
  fig, ax = plt.subplots(figsize=(7, 6))
  disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
  disp.plot(include_values=True, cmap='Blues', ax=ax, xticks_rotation='horizontal')
  ax.set_title(title)
  ax.set_xlabel('Predicted Label')
  ax.set_ylabel('True Label')
  plt.show()
  return cm

# Function that plots the RocCurve for the given test-set
def roccurve_AD(y_true, y_scores):

  fpr, tpr, thresholds = roc_curve(y_true, y_scores)
  auc_score = auc(fpr, tpr)
  # plt.figure(figsize=(7, 6))
  plt.plot(fpr, tpr, label=f'ROC curve (AUC = {auc_score:.2f})')
  plt.plot([0,1],[0,1],'g--')
  plt.legend()
  plt.xlabel('False Positive Rate')
  plt.ylabel('True Positive Rate')
  plt.title("AUC(ROC curve)")  
  plt.grid(color='black', linestyle='-', linewidth=0.5)
  plt.show()
  print("\n")

# Function that Prints and plots a lot of Metrics
def calculate_metrics_AD(X_true, y_true, pred_scores, y_pred):

  # Confusion matrix
  print('Confusion Matrix:\n')
  cm = conf_matrix_AD(y_true, y_pred)
      
  # Calculate the AUC-ROC
  auc_roc = roc_auc_score(y_true, pred_scores)
  
  # ROC_Curve
  roccurve_AD(y_true, pred_scores)

  # Calculate the accuracy
  accuracy = accuracy_score(y_true, y_pred)

  # Calculate the precision
  precision = precision_score(y_true, y_pred)

  # Calculate the recall
  recall = recall_score(y_true, y_pred)

  # Calculate the F1 score
  f1 = f1_score(y_true, y_pred)

  # Print the results
  print(f'AUC-ROC: {auc_roc:.4f}')
  print(f'Accuracy: {accuracy:.4f}')
  print(f'F1 Score: {f1:.4f}')
  print(f'Precision: {precision:.4f}')
  print(f'Recall: {recall:.4f}')

  #return cm, auc_roc, accuracy, f1, precision, recall
  
def split_data(X, y, chosen_class=0, percentage=0.8):
    if chosen_class not in np.unique(y):
        raise ValueError("Chosen class not found in the target variable.")
  
    # Filter X for the chosen class: y = class
    X_c = X[y == chosen_class]
    y_c = y[y == chosen_class]

    # Randomly shuffle the rows of X_c and y_c
    random_indices = np.random.permutation(len(X_c))
    X_c = X_c[random_indices]
    y_c = y_c[random_indices]

    # Split X_c and y_c into training and testing sets
    train_size = int(percentage * len(X_c))
    X_train = X_c[:train_size]
    y_train = y_c[:train_size]
    X_test_n = X_c[train_size:]
    y_test_n = y_c[train_size:]

    # Filter X for other classes: y != class
    X_new_a = X[y != chosen_class]
    y_new_a = y[y != chosen_class]

    # Randomly select rows from X_new_a and y_new_a to create X_test_a and y_test_a
    select = len(X_test_n)
    random_indices = np.random.choice(len(X_new_a), size=select, replace=False)
    X_test_a = X_new_a[random_indices]
    y_test_a = y_new_a[random_indices]

    # Combine X_test_n, X_test_a, y_test_n, and y_test_a to create X_test and y_test
    X_test = np.concatenate((X_test_n, X_test_a), axis=0)
    y_test = np.concatenate((y_test_n, y_test_a), axis=0)
    
    return X_train, X_test, X_test_n, X_test_a, y_train, y_test, y_test_n, y_test_a


**Park's get the AUC with the mean of mse of each class.**

In [5]:
def split_data(X, y, chosen_class=0, percentage=0.8):
    if chosen_class not in np.unique(y):
        raise ValueError("Chosen class not found in the target variable.")
  
    # Filter X for the chosen class: y = class
    X_c = X[y == chosen_class]
    y_c = y[y == chosen_class]

    # Randomly shuffle the rows of X_c and y_c
    random_indices = np.random.permutation(len(X_c))
    X_c = X_c[random_indices]
    y_c = y_c[random_indices]

    # Split X_c and y_c into training and testing sets
    train_size = int(percentage * len(X_c))
    X_train = X_c[:train_size]
    y_train = y_c[:train_size]
    X_test_n = X_c[train_size:]
    y_test_n = y_c[train_size:]

    # Filter X for other classes: y != class
    X_new_a = X[y != chosen_class]
    y_new_a = y[y != chosen_class]

    # Randomly select rows from X_new_a and y_new_a to create X_test_a and y_test_a
    select = len(X_test_n)
    random_indices = np.random.choice(len(X_new_a), size=select, replace=False)
    X_test_a = X_new_a[random_indices]
    y_test_a = y_new_a[random_indices]

    # Combine X_test_n, X_test_a, y_test_n, and y_test_a to create X_test and y_test
    X_test = np.concatenate((X_test_n, X_test_a), axis=0)
    y_test = np.concatenate((y_test_n, y_test_a), axis=0)
    
    return X_train, X_test, X_test_n, X_test_a, y_train, y_test, y_test_n, y_test_a

def kernelpca_OCC(X_train, X_test_n, X_test_a, n_components=None, gamma=0.01, kernel="rbf", alpha=0.1):

  kernel_pca = KernelPCA(n_components=n_components, kernel=kernel, gamma=gamma, fit_inverse_transform=True, alpha=alpha)
  kernel_pca.fit(X_train)

  X_train_reconstructed = kernel_pca.inverse_transform(kernel_pca.transform(X_train))
  X_test_n_reconstructed = kernel_pca.inverse_transform(kernel_pca.transform(X_test_n))
  X_test_a_reconstructed = kernel_pca.inverse_transform(kernel_pca.transform(X_test_a))

  n_mse = np.square(np.subtract(X_test_n,X_test_n_reconstructed)).mean(axis=1)
  a_mse = np.square(np.subtract(X_test_a,X_test_a_reconstructed)).mean(axis=1)

  return n_mse, a_mse

def AUC_OC(n_mse, a_mse):
  
  y_true = np.array([0]*len(n_mse)+[1]*len(a_mse))
  y_score = np.r_[n_mse,a_mse]

  fpr, tpr, _ = roc_curve(y_true, y_score)
  auc_measure = auc(fpr,tpr)

  return auc_measure


def Kernel_PCA_AUC(X, y, split_percentage=0.8, one_class=-1, kernel_analysis=False):
  # Get the unique classes
  all_classes = np.unique(y)
  print("\nAll different classes: ", all_classes)
  
  # Count the number of unique classes in y
  number_of_classes = len(all_classes)
  print("\nNumber of classes: ", number_of_classes)

  if (one_class==-1) or (one_class not in all_classes):
    # do for all classes
    for oc in range(number_of_classes):
      X_train, X_test, X_test_n, X_test_a, y_train, y_test, y_test_n, y_test_a = split_data(X, y, oc, split_percentage)
      if kernel_analysis==False:
        n_mse, a_mse = kernelpca_OCC(X_train, X_test_n, X_test_a)
        auc = AUC_OC(n_mse, a_mse)
        print("\nFor the class: ", oc)
        print("\nAUC: ", auc)
      else:
        best_auc = 0.0
        ooptimum_gamma = 0
        for gamma in np.logspace(-10, 10, num=20, base=2):
          n_mse, a_mse = kernelpca_OCC(X_train, X_test_n, X_test_a, gamma=gamma)
          new_auc = AUC_OC(n_mse, a_mse)
          # print("\nFor the class: ", oc)
          # print("\n gamma: ", gamma)
          # print("\nAUC: ", new_auc)
          if new_auc > best_auc :
            best_auc = new_auc
            optimum_gamma = gamma
        print("=====================")
        print("\nFor the class: ", oc)
        print("\nOptimum gamma: ", optimum_gamma)
        print("\nBest AUC: ", best_auc)
        print("=====================")
  else:
      # do for the chosen 
      X_train, X_test, X_test_n, X_test_a, y_train, y_test, y_test_n, y_test_a = split_data(X, y, one_class, split_percentage)
      if kernel_analysis==False:
        n_mse, a_mse = kernelpca_OCC(X_train, X_test_n, X_test_a)
        auc = AUC_OC(n_mse, a_mse)
        print("\nFor the class: ", one_class)
        print("\nAUC: ", auc)
      else:
        # do for the chosen 
        for gamma in np.logspace(-10, 1, num=10, base=2):
            n_mse, a_mse = kernelpca_OCC(X, y, split_percentage, one_class, gamma=gamma)
            new_auc = AUC_OC(n_mse, a_mse)
            print("\nFor the class: ", one_class)
            print("\n gamma: ", gamma)
            print("\nAUC: ", new_auc)
            if new_auc > best_auc :
              best_auc = new_auc
              optimum_gamma = gamma
        print("=====================")
        print("\nFor the class: ", one_class)
        print("\nOptimum gamma: ", optimum_gamma)
        print("\nBest AUC: ", best_auc)
        print("=====================")